In [31]:
from google.colab import drive

drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [32]:
!pip install datasets transformers accelerate flash-attention flash_attn


In [33]:
from datasets import load_dataset

In [34]:
dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/codexglue/test.jsonl")

In [35]:
dataset["train"][0]

{'signature': 'def debug(user, message):',
 'body': 'message_user(user, message, constants.DEBUG)<EOL>',
 'docstring': 'Adds a message with the ``DEBUG`` level.\n\n:param user: User instance\n:param message: Message to show',
 'id': 'f4:m0'}

In [36]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cuda")
#eos_token = tokenizer.convert_tokens_to_ids("\n\n")
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1.5", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1.5")
#tokenizer.pad_token = "\n\n"


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [37]:
from tqdm import tqdm
i = 0
outputs = []
for data in tqdm(dataset["train"]):
    if i == 100:
        break
    input_text = f'''Write a python method for this signature and docstring, output only python code and nothing else
{data["signature"]}
"""
{data["docstring"]}
"""
'''
    inputs = tokenizer(input_text, return_tensors="pt")#, padding=True, truncation=True, max_length = 512, return_attention_mask=True)

    output = model.generate(**inputs, max_new_tokens=120)#, do_sample = True, temperature = 1.0, top_k = 50, early_stopping = True, num_beams = 5, num_return_sequences=1)
    text = tokenizer.batch_decode(output)[0]
    #print(text)
    #print("="*80)
    outputs.append(text[len(input_text):])

    i += 1

with open("/content/gdrive/MyDrive/predictions_python.txt", "w") as file:
    for text in outputs:
        file.write(text.replace("\n", "").replace("\t", "") + "\n")


  0%|          | 100/20000 [06:48<22:34:46,  4.08s/it]


In [38]:
dataset_kotlin = load_dataset("json", data_files="/content/gdrive/MyDrive/test_kotlin.json")

In [39]:
dataset_kotlin["train"][0]

{'body': '    a.x = 42\n    return value\n}\n',
 'docstring': '',
 'signature': 'inline fun bar(value: Int, a: A): Int {\n'}

In [40]:
i = 0
outputs = []
for data in tqdm(dataset_kotlin["train"]):
    if i == 100:
        break
    input_text = f'''Write a kotlin method for this signature and docstring, output only kotlin code and nothing else
{data["docstring"]}
{data["signature"]}
'''
    inputs = tokenizer(input_text, return_tensors="pt")#, padding=True, truncation=True, max_length = 512, return_attention_mask=True)

    output = model.generate(**inputs, max_new_tokens=120)#, do_sample = True, temperature = 1.0, top_k = 50, early_stopping = True, num_beams = 5, num_return_sequences=1)
    text = tokenizer.batch_decode(output)[0]
    #print(text)
    #print("="*80)
    outputs.append(text[len(input_text):])

    i += 1

with open("/content/gdrive/MyDrive/predictions_kotlin.txt", "w") as file:
    for text in outputs:
        file.write(text.replace("\n", "").replace("\t", "") + "\n")


  1%|          | 100/14203 [06:32<15:22:09,  3.92s/it]
